In [1]:
import MKLpy
from MKLpy.metrics import pairwise
from MKLpy.algorithms import AverageMKL, EasyMKL, KOMD#KOMD is not a MKL algorithm but a simple kernel machine like the SVM
#evaluate the solution
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import roc_curve, auc
from MKLpy.metrics import pairwise
from MKLpy.preprocessing import normalization, rescale_01
from MKLpy.preprocessing import kernel_normalization

from sklearn.metrics.pairwise import rbf_kernel as RBF
import os
import pandas as pd
import numpy as np
import pickle as pkl

In [6]:

symbolIdx = 2  # pick one of the symbols

# do a join to get the location
# symbolLocation = "/".join((finalLocation, symbols[symbolIdx]))
# # get the features now

selectionLoc = os.path.join(hardDrivesLoc, selection)

# ''' location of WorkDrive'''
# dataList = [s for s in os.listdir(selectionLoc) if s.startswith('Dat')]
# path = 'MKL_Experiments'
MKLExpPath = os.path.join(os.path.join(hardDrivesLoc, selection, dataList[1]), 'MKL_Experiments')
MKLSymbolPath = os.path.join(MKLExpPath, symbols[symbolIdx])
MKLSymbolKernelsPath = "/".join((MKLSymbolPath, 'Kernels'))

SymbolCommonPaths = mkl_base.open_pickle_file(MKLSymbolPath, 'LocDictsListCorrect.pkl')
uniqueTrainingKeys = np.unique([i[1] for i in SymbolCommonPaths])

NameError: name 'hardDrivesLoc' is not defined

In [ ]:

kernelDates = [file.split("_")[0] for file in os.listdir('/media/ak/WorkDrive/Data/MKL_Experiments/PRU.L/Kernels/')]

                

In [ ]:
kernelDates[0]

In [ ]:
keys=list(SymbolCommonPaths.keys())
type(keys)
# for idx, key in enumerate(keys):
#     print(idx)
#     print(key)
#     kernelFileName = "/".join((MKLSymbolKernelsPath, "".join((key[0], "_Kernels.pkl"))))
#     checkDir(kernelFileName)
#     print(kernelFileName)
#     print(SymbolCommonPaths[key])
i=kernelDates[0]
result=[]
for j in keys:
    if i in j:
        result.append(j)

print(result)

In [ ]:
print(SymbolCommonPaths[keys[920]])
kernelFileName = "/".join((MKLSymbolKernelsPath, "".join((i, "_Kernels.pkl"))))
checkDir(kernelFileName)

In [ ]:
featuresTupleFile=pkl.load(open(SymbolCommonPaths[result[1]][0], "rb"),encoding='latin1')


In [ ]:
labelsDf  = pd.read_csv(SymbolCommonPaths[result[1]][1])


In [ ]:
''' pop the labels out'''
labels = labelsDf['label_PrMov__window_5__thres_arbitrary__0.1']
''' and same with features'''
dfFeatures = pd.concat([featuresTupleFile[0], featuresTupleFile[1], \
                            featuresTupleFile[2], featuresTupleFile[3]], axis=1, sort=False).fillna(0)

In [ ]:


'''dataframe of Features and Labels - X and Y'''
dfXY = pd.concat([dfFeatures, labels], axis=1, sort='False').dropna()
labelName = str(dfXY.columns[dfXY.columns.str.contains(pat='label')].values[0])
''' drop the labels from the features'''
dfX = dfXY.drop(columns=[labelName])
arrX = np.array(dfX)
''' feature normalisation'''
# feature scaling in [0,1] - X = rescale_01(arrX)
X = normalization(rescale_01(arrX))
y = dfXY[dfXY.columns[dfXY.columns.str.contains(pat='label')]].iloc[:, 0]
''' returns features, labels'''

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
KLtr = [pairwise.homogeneous_polynomial_kernel(X, degree=d) for d in range(4)]
''' Compute RBF Kernels'''
gamma_range = np.logspace(-9, 3, 13)
ker_list = [RBF(X, gamma=g) for g in gamma_range]

## should get the components to have a shape X.shape x y.shape

In [ ]:
## try some bullshit test ###
clf = AverageMKL().fit(KLtr,y) #a wrapper for averaging kernels
print(clf.weights) #print the weights of the combination of base kernels
print ('training EasyMKL...for polynomials and RBF', end='')
clfEasy = EasyMKL(lam=0.1).fit(KLtr,y)#combining kernels with the EasyMKL algorithm

In [ ]:
## and test the bullshit of ker_list ##
clfRBF = EasyMKL(lam=0.1).fit(ker_list,y) 

In [ ]:
KernelsTest = [KLtr, ker_list]
testFileName = 'KernelTestPickle.pkl'
PapersPath = os.getcwd()
pkl.dump(KernelsTest, open("/".join((PapersPath, testFileName)), "wb"))

In [ ]:
loadPickleKernels = pkl.load(open("/".join((PapersPath, testFileName)), "rb"))

In [ ]:
KLtrloaded = loadPickleKernels[1]
ker_list_loaded= loadPickleKernels[1]

In [ ]:
## try some bullshit test ###
clfloaded = AverageMKL().fit(KLtrloaded,y) #a wrapper for averaging kernels
clfEasyloaded = EasyMKL(lam=0.1).fit(KLtrloaded,y)#combining kernels with the EasyMKL algorithm
print('test complete')

In [ ]:
 ### and train 3 classifiers ###
    clf = AverageMKL().fit(KLtr,ytr) #a wrapper for averaging kernels
    print(clf.weights) #print the weights of the combination of base kernels
    print ('training EasyMKL...for polynomials and RBF', end='')
    clfEasy = EasyMKL(lam=0.1).fit(KLtr,ytr)#combining kernels with the EasyMKL algorithm
    clfRBF = EasyMKL(lam=0.1).fit(ker_list,ytr)

In [ ]:
if __name__ == '__main__':
    trainingKey =1 
    keyDate = list(SymbolCommonPaths)[trainingKey]
    
    

    for trainingKey, keyDate in enumerate(list(SymbolCommonPaths)):

        print(trainingKey, keyDate[1])
        Xtr, ytr = featureCreation(idxKey=trainingKey,
                                   locDict=SymbolCommonPaths)  # need to refactor this to read from the mkl_base
        if Xtr.shape[0] == ytr.shape[0]:
            print(list(SymbolCommonPaths.keys())[trainingKey][0])
            kernelFileName = "/".join((MKLSymbolKernelsPath, "".join((keyDate[0], "_Kernels.pkl"))))
            loadPickleKernels = pkl.load(open(kernelFileName, "rb"))
            print('doing the kernels bit')
            KLsimple = loadPickleKernels[0]
            KLrbf = loadPickleKernels[1]
            clf = AverageMKL().fit(KLsimple, ytr)  # a wrapper for averaging kernels
            clfEasy = EasyMKL(lam=0.1).fit(KLsimple, ytr)#combining kernels with the EasyMKL algorithm
            clfRBF = EasyMKL(lam=0.1).fit(KLrbf, ytr)
            print('------')
            print('finished MKL Fitting- Now will test Predictions')
            print('Average Kernel Testing')
            y_pred = clf.predict(KLsimple)  # predictions
            y_score = clf.decision_function(KLsimple)  # rank
            accuracy = accuracy_score(ytr, y_pred)
            fprAverage, tprAverage, thresholdsAverage = roc_curve(ytr.ravel(), y_score.ravel())
            roc_auc = auc(fprAverage, tprAverage)
            print('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))
        else:
            print('Shapes dont match.')
            continue

In [ ]:
trainingKey =1 
keyDate = list(SymbolCommonPaths)[trainingKey]
print(trainingKey, keyDate[1])
Xtr, ytr = featureCreation(idxKey=trainingKey,
                                   locDict=SymbolCommonPaths)  # need to refactor

In [ ]:
Xtr.shape[0] == ytr.shape[0]

In [ ]:
print(list(SymbolCommonPaths.keys())[trainingKey][0])
kernelFileName = "/".join((MKLSymbolKernelsPath, "".join((keyDate[1], "_Kernels.pkl"))))
loadPickleKernels = pkl.load(open(kernelFileName, "rb"))
print('doing the kernels bit')
KLsimple = loadPickleKernels[0]
KLrbf = loadPickleKernels[1]

In [ ]:
clf = AverageMKL().fit(KLsimple, ytr)  # a wrapper for averaging kernels
clfEasy = EasyMKL(lam=0.1).fit(KLsimple, ytr)#combining kernels with the EasyMKL algorithm
            
clfRBF = EasyMKL(lam=0.1).fit(KLrbf, ytr)

 ######################### 19 April 2021 ###############################~

In [2]:
import pickle as pkl
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np
import pickle

import os
import pickle
import pandas as pd
import sys
import sys
from memory_profiler import profile
import pandas as pd

sys.path.append('/home/ak/Documents/Research/PaperCode/singlekernelclf/')
import jsonpickle
from MKLpy.preprocessing import normalization, rescale_01
from MKLpy.metrics import pairwise
from MKLpy import generators
from MKLpy.algorithms import AverageMKL, EasyMKL
from sklearn.svm import SVC
from collections import OrderedDict
import os
import torch
from MKLpy.multiclass import OneVsRestMKLClassifier, OneVsOneMKLClassifier

from MKLpy.algorithms import AverageMKL, EasyMKL, \
    KOMD  # KOMD is not a MKL algorithm but a simple kernel machine like the SVM

from fileutils import new_feature_utils as nfu
from fileutils.new_feature_utils import CreateMarketFeatures


In [3]:

def open_pickle_filepath(pickle_file):
    pickle_to_file = pickle.load(open(pickle_file, "rb"), encoding='latin1')

    return pickle_to_file


In [4]:
oosFolder  = '/media/ak/DataOnly/ExperimentCommonLocs/OOSPredictions/'
labelsAnalysisResults = '/media/ak/DataOnly/ExperimentCommonLocs/LabelAnalysisResults/'
crossValResults = '/media/ak/DataOnly/ExperimentCommonLocs/CrossValidationResults'

In [5]:
jointFeatureLocation = os.path.join('/media/ak/DataOnly/ExperimentCommonLocs', "JointLocationsDicts")

In [6]:
sorted(os.listdir(jointFeatureLocation))

['AAL.L_hmm_date:_20170116_feature_date_20170831_label_id:_1_ProcessedData.pkl',
 'AAL.L_hmm_date:_20170116_feature_date_20170831_label_id:_2_ProcessedData.pkl',
 'AAL.L_hmm_date:_20170116_feature_date_20170831_label_id:_3_ProcessedData.pkl',
 'AAL.L_hmm_date:_20170116_feature_date_20170831_label_id:_4_ProcessedData.pkl',
 'AAL.L_hmm_date:_20170116_feature_date_20170831_label_id:_5_ProcessedData.pkl',
 'AAL.L_hmm_date:_20170116_feature_date_20170831_label_id:_6_ProcessedData.pkl',
 'AAL.L_hmm_date:_20170116_feature_date_20170831_label_id:_7_ProcessedData.pkl',
 'AAL.L_hmm_date:_20170117_feature_date_20170831_label_id:_1_ProcessedData.pkl',
 'AAL.L_hmm_date:_20170117_feature_date_20170831_label_id:_2_ProcessedData.pkl',
 'AAL.L_hmm_date:_20170117_feature_date_20170831_label_id:_3_ProcessedData.pkl',
 'AAL.L_hmm_date:_20170117_feature_date_20170831_label_id:_4_ProcessedData.pkl',
 'AAL.L_hmm_date:_20170117_feature_date_20170831_label_id:_5_ProcessedData.pkl',
 'AAL.L_hmm_date:_20170117_f

In [14]:
test_cross_val_file = os.path.join(crossValResults, crossValFiles[1])

In [ ]:
pd.DataFrame.from_dict(open_pickle_filepath(test_cross_val_file)

In [32]:
for file in crossValFiles:
    symbol = file.split("_")[0]
    label_idx = file.split("_")[5]
    feature_label_date = file.split("_")[4]
    print(open_pickle_filepath(os.path.join(crossValResults,file)))


{'C': 1, 'lam': 0.2, 'score': 0.5666666666666667, 'scores': [0.3333333333333333, 0.5, 0.5, 0.5, 1.0]}
{'C': 0.1, 'lam': 0.2, 'score': 0.5333333333333333, 'scores': [0.6666666666666666, 0.0, 0.5, 0.5, 1.0]}
{'C': 0.1, 'lam': 0.2, 'score': 0.4666666666666666, 'scores': [0.3333333333333333, 0.5, 0.5, 0.5, 0.5]}
{'C': 0.1, 'lam': 0.2, 'score': 0.9970297029702969, 'scores': [1.0, 1.0, 0.995049504950495, 0.995049504950495, 0.995049504950495]}
{'C': 0.1, 'lam': 0.2, 'score': 0.9928253603287136, 'scores': [0.993485342019544, 0.993485342019544, 0.990228013029316, 0.9934640522875817, 0.9934640522875817]}
{'C': 0.1, 'lam': 0.2, 'score': 0.4666666666666666, 'scores': [0.3333333333333333, 0.5, 0.5, 0.5, 0.5]}
{'C': 0.1, 'lam': 0.2, 'score': 0.36666666666666664, 'scores': [0.3333333333333333, 0.5, 0.5, 0.5, 0.0]}
{'C': 0.1, 'lam': 0.2, 'score': 0.9957713338856669, 'scores': [0.9953051643192489, 0.9953051643192489, 0.9953051643192489, 0.9976470588235294, 0.9952941176470588]}
{'C': 0.1, 'lam': 0.2, 's

In [1]:
import multiprocessing as mp
mp.cpu_count()

16

In [10]:
joint_loc_files = os.listdir(jointFeatureLocation)

In [14]:
jointFeatureIdx = 0
file_path = os.path.join(jointFeatureLocation, joint_loc_files[jointFeatureIdx])
file_path

'/media/ak/DataOnly/ExperimentCommonLocs/JointLocationsDicts/AZN.L_hmm_date:_20170802_feature_date_20170929_label_id:_3_ProcessedData.pkl'

In [18]:
file_path_dict = open_pickle_filepath(file_path)
hmm_keys = list(open_pickle_filepath(file_path).keys())

In [21]:
print(hmm_keys[1])

20170117


In [26]:
test_keys_dict = dict()
for key in hmm_keys:
    test_keys_dict[key] = (sorted(file_path_dict[key].keys()))

In [27]:
test_keys_dict

{'20170116': ['20170117',
  '20170118',
  '20170119',
  '20170120',
  '20170123',
  '20170124',
  '20170125',
  '20170126',
  '20170127',
  '20170130',
  '20170131',
  '20170703',
  '20170704',
  '20170705',
  '20170706',
  '20170801',
  '20170802',
  '20170803',
  '20170804',
  '20170807',
  '20170808',
  '20170809',
  '20170810',
  '20170811',
  '20170814',
  '20170815',
  '20170816',
  '20170817',
  '20170818',
  '20170821',
  '20170822',
  '20170823',
  '20170824',
  '20170825',
  '20170829',
  '20170830',
  '20170831',
  '20170901',
  '20170904',
  '20170905',
  '20170906',
  '20170907',
  '20170908',
  '20170911',
  '20170912',
  '20170913',
  '20170914',
  '20170915',
  '20170918',
  '20170919',
  '20170920',
  '20170921',
  '20170922',
  '20170925',
  '20170926',
  '20170927',
  '20170928',
  '20170929'],
 '20170117': ['20170118',
  '20170119',
  '20170120',
  '20170123',
  '20170124',
  '20170125',
  '20170126',
  '20170127',
  '20170130',
  '20170131',
  '20170703',
  '201707